# CSC421 Fall 2023 Assignment 2
### Author: George Tzanetakis

This notebook is based on the supporting material for topics covered in **Chapter 7 - Logical Agents** and **Chapter 8 - First-Order Logic** from the book *Artificial Intelligence: A Modern Approach.* You can consult and modify the textbook code provided in logic.py and logic.ipynb for completing the assignment questions. The first 5 questions do NOT rely on the provided aima code so you can complete them just using basic Python. Questions 6 and 7 do require the use of the aima-code repository.

You are welcome and actually it can be educational to look at the code at the aima-code repository as well as other code resources you can find on the web. However, make sure you understand any code that you incoporate.

Each question is worth 1% of the total grade. The advanced questions require significantly more
effort than the basic and expected questions. The assignment structure is as follows:

1. Propositional Logic (Basic) - simple infix evaluator for 0, 1 and logical operators
2. Propositional Logic (Basic) - adding variables and bindings to evaluator
3. Propositional Logic (Expected) - recursive infix evaluator for propositional logic
4. Propositional Logic (Expected) - usage of evaluator to evaluate some example logic expressions
5. Propositional Logic (Advanced) - model checking for the prefix evaluator
6. Propositional Logic(Basic) - simple KB using aima code with model checking and theorem proving
7. First-Order Logic (Basic) - kinship domain using FolKB for the Simpsons
8. First-Order Logic (Expected) - kinship domain using Prolog for the Simpsons
9. First-Order Logic (Expected) - LegoWorld
10. First-Order Logic (Advanced) - NLP for LegoWorld or webscraping a KB for music  

```
Birds can fly, unless they are penguins and ostriches, or if they happen
to be dead, or have broken wings, or are confined to cages, or have their
feet stuck in cement, or have undergone experiences so dreadful as to render
them psychologically incapable of flight

Marvin Minsky
```

# Introduction - Parsing and evaluating prefix logic expressions  

In this assignment your task is to incrementally create a parser and evaluator for prefix logic expressions as well as implement simple model checking.


# Question 1 (Basic)

Your first task will be to write a simple evaluator of prefix logic expressions with constants. In prefix notation the operator precedes the operands and no operands are required. For example 5+3 in prefix notation is written + 5 3 or 5 * 2 + 3 would be written + * 5 2 3 or + * 5 2 * 3 4 is equivalent to (5 * 2) + (3 * 4).

As a first step we will consider very simple expressions with one operator and two constant operands. We will use 0 for false and 1 for true. The following logical connectives should be implemented (see Figure 7.8 in your book) (notice that for now there is no negation symbol ~):

1. &    (and),
3. |    (or),
4. =>   (implication)
5. <=>  (biconditional)

Example expressions:
```
& 1 0  
=> 0 1
<=> 1 1
```

Your function should take as input a string with the prefix expression and return the result of evaluating the expression (basically a 1 for true and 0 for false). You can split a string to a list using .split[' ']. For this part of the assignment you only evaluate expressions with two constant operands i.e no nested/recursive expressions.

In [ ]:
a = '& 1 0'
print(a.split(' '))

['&', '1', '0']


In [85]:
# YOUR CODE GOES HERE
def evaluate_prefix_expression(expression):
    tokens = expression.split()

    # Operator mapping to their corresponding functions
    operators = {
        '&': lambda x, y: x & y,
        '|': lambda x, y: x | y,
        '=>': lambda x, y: (not x) or y,
        '<=>': lambda x, y: (x and y) or (not x and not y)
    }

    operator = tokens[0]
    operand1 = int(tokens[1])
    operand2 = int(tokens[2])

    if operator not in operators:
        return "Invalid operator"

    result = operators[operator](operand1, operand2)

    result = 1 if result else 0

    return result

# Example usage:
exp1 = '& 1 0'
exp2 = '| 0 1'
exp3 =  '=> 0 1'
exp4 = '<=> 1 1'

print(evaluate_prefix_expression(exp1))
print(evaluate_prefix_expression(exp2))
print(evaluate_prefix_expression(exp3))
print(evaluate_prefix_expression(exp4))


0
1
1
1


# Question 2 (Basic)

Your next task is to implement variables and bindings for your propositional logic evaluator. In this version in addition to constants (0 and 1) you also can have variables which are strings with associated values provided in a dictionary. You still only consider two operands and one operator (no nesting). For example in the code below
the three expressions are equivalent. Your function should take as arguments the expression to be evaluated as a string and the dictionary with the variable bindings. In addition you need to add the ~ (not) operator. To do so for each variable in the dictionary add a not version. For example if 'a' in the dictionary has a value of 1 the '~a' in the dictionary should have a value of 0. Notice that the not symbol is part of the string and it is NOT separated by a space.



In [ ]:
d = {'foo': 0, 'b': 1}
print(d)
expr1 = '& 0 1'
expr2 = '& foo 1'
expr3 = '& foo ~b'

{'foo': 0, 'b': 1}


In [86]:
# YOUR CODE GOES HERE
def evaluate_prefix_expression_with_variables(expression, bindings):
    tokens = expression.split()

    operators = {
        '&': lambda x, y: x & y,
        '|': lambda x, y: x | y,
        '=>': lambda x, y: (not x) or y,
        '<=>': lambda x, y: (x and y) or (not x and not y)
    }

    def evaluate_token(token):
        if token.isdigit():
            return int(token)
        elif token.startswith('~'):
            return 1 - bindings[token[1:]]  # Get the not version from the bindings
        else:
            return bindings[token]

    operator = tokens[0]
    operand1 = evaluate_token(tokens[1])
    operand2 = evaluate_token(tokens[2])

    if operator not in operators:
        return "Invalid operator"

    result = operators[operator](operand1, operand2)
    result = 1 if result else 0

    return result

# Example usage:
d = {'foo': 0, 'b': 0}
print(d)
expr1 = '& ~b 1'
expr2 = '& foo 1'
expr3 = '& ~foo ~b'

print(evaluate_prefix_expression_with_variables(expr1, d))
print(evaluate_prefix_expression_with_variables(expr2, d))
print(evaluate_prefix_expression_with_variables(expr3, d))



{'foo': 0, 'b': 0}
1
0
1


# Question 3 (Expected)


The following code is a recursive evaluator for prefix arithmetic expressions. It assumes that there are always two operands either an integer or a prefix expression starting with an operator (addition or multiplication). It is a good idea to go through this function carefully by hand to understand how the recursion works.

Informed by your understanding of the arithmetic recursive_eval function your task is to write function to implement a recursive prefix logic evaluator. Your evaluator should also support variables bindings using a dictionary as in the previous question.

Example expressions:
```
& 1 & 1 a   
=> 0 & b ~alice  
<=> foo 1
```

In [ ]:
def recursive_eval(l):
    head, tail = l[0], l[1:]
    if head in ['+', '*']:
        val1, tail = recursive_eval(tail)
        val2, tail = recursive_eval(tail)
        if head == '+':
            return (int(val1)+int(val2), tail)
        elif head == '*':
            return (int(val1)*int(val2), tail)
    # operator is a number
    else:
        return (int(head),tail)

def prefix_eval(input_str):
    input_list = input_str.split(' ')
    res, tail = recursive_eval(input_list)
    return res

print(prefix_eval('1'))
print(prefix_eval('+ 1 2'))
print(prefix_eval('+ 1 * 2 3'))
print(prefix_eval('+ * 5 2 * 3 + 1 5'))

1
3
7
28


In [ ]:
# YOUR CODE GOES HERE

def evaluate_prefix_recursive(binding, expression):
    def replace_variables(token):
        if token.startswith('~'):
            return str(int(not binding[token[1:]]))
        return str(binding[token]) if token in binding else token

    tokens = expression.split()
    operators = {
        '&': lambda x, y: x and y,
        '|': lambda x, y: x or y,
        '=>': lambda x, y: 1 if x <= y else 0,
        '<=>': lambda x, y: 1 if x == y else 0
    }

    def evaluate(tokens):
        if not tokens:
            return None

        token = tokens.pop(0)

        if token in operators:
            value1 = evaluate(tokens)
            value2 = evaluate(tokens)
            return operators[token](int(value1), int(value2))

        return int(replace_variables(token))

    result = evaluate(tokens)
    return result

# Test cases and variable bindings
expression_a = '& 1 & 1 a'
expression_b = '=> 0 & b ~alice'
expression_c = '<=> foo 1'
bindings = {'foo': 0, 'alice': 0, 'a': 0, 'b': 0}

print(evaluate_prefix_recursive(bindings, expression_a))  # Output: 0
print(evaluate_prefix_recursive(bindings, expression_b))  # Output: 1
print(evaluate_prefix_recursive(bindings, expression_c))  # Output: 0




0
1
0


# QUESTION 4 (EXPECTED)


Using the recursive prefix evaluator you defined in the previous question
answer the following question (you will need to convert the exressions below
to prefix). You can use multiple string assignments to assemble more complicated
sentences into one big string:


Let A be the formula:

\begin{equation}
  (( p_{1} \rightarrow (p2 \land p_{3})) \land ((\neg p_{1})
  \rightarrow (p_{3} \land p_{4})))
\end{equation}

Let B be the formula:

\begin{equation}
  (( p_{3} \rightarrow (\neg p_{6})) \land ((\neg
  p_{3}) \rightarrow (p_{4} \rightarrow p_{1})))  
\end{equation}

Let C be the formula:

\begin{equation}
  ((\neg(p2 \land p_{5})) \land (p2 \rightarrow p_{5}))
\end{equation}

Let D be the formula:

\begin{equation}
  (\neg (p_{3} \rightarrow p_{6}))
\end{equation}

Evaluate the formulate E:
\begin{equation}
  (( A \land (B \land C)) \rightarrow D)
\end{equation}

under the true assignment $I_{1}$, where $I_{1}(p_{1}) = I_{1}(p_{3}) = I_{1}(p_{5}) = false$
and $I_{1}(p2) = I_{1}(p_{4}) = I_{1}(p_{6}) = true$ as well as under the truth assignment
$I_{2}$, where $I_{2}(p_{1}) = I_{2}(p_{3}) = I_{2}(p_{5}) = true$ and
$I_{2}(p_{2})=I_{2}(p_{4})=I_{2}(p_{6}) = false$.


In [ ]:
## YOUR CODDE GOES HERE



# Question 3 (Expected)


The following code is a recursive evaluator for prefix arithmetic expressions. It assumes that there are always two operands either an integer or a prefix expression starting with an operator (addition or multiplication). It is a good idea to go through this function carefully by hand to understand how the recursion works.

Informed by your understanding of the arithmetic recursive_eval function your task is to write function to implement a recursive prefix logic evaluator. Your evaluator should also support variables bindings using a dictionary as in the previous question.

Example expressions:
```
& 1 & 1 a   
=> 0 & b ~alice  
<=> foo 1
```

In [ ]:
def recursive_eval(l):
    head, tail = l[0], l[1:]
    if head in ['+', '*']:
        val1, tail = recursive_eval(tail)
        val2, tail = recursive_eval(tail)
        if head == '+':
            return (int(val1)+int(val2), tail)
        elif head == '*':
            return (int(val1)*int(val2), tail)
    # operator is a number
    else:
        return (int(head),tail)

def prefix_eval(input_str):
    input_list = input_str.split(' ')
    res, tail = recursive_eval(input_list)
    return res

print(prefix_eval('1'))
print(prefix_eval('+ 1 2'))
print(prefix_eval('+ 1 * 2 3'))
print(prefix_eval('+ * 5 2 * 3 + 1 5'))

1
3
7
28


In [89]:
# YOUR CODE GOES HERE

def evaluate_prefix_recursive(binding, expression):
    def replace_variables(token):
        if token.startswith('~'):
            return str(int(not binding[token[1:]]))
        return str(binding[token]) if token in binding else token

    tokens = expression.split()
    operators = {
        '&': lambda x, y: x and y,
        '|': lambda x, y: x or y,
        '=>': lambda x, y: 1 if x <= y else 0,
        '<=>': lambda x, y: 1 if x == y else 0
    }

    def evaluate(tokens):
        if not tokens:
            return None

        token = tokens.pop(0)

        if token in operators:
            value1 = evaluate(tokens)
            value2 = evaluate(tokens)
            return operators[token](int(value1), int(value2))

        return int(replace_variables(token))

    result = evaluate(tokens)
    return result

# Test cases and variable bindings
expression_a = '& 1 & 1 a'
expression_b = '=> 0 & b ~alice'
expression_c = '<=> foo 1'
bindings = {'foo': 0, 'alice': 0, 'a': 0, 'b': 0}

print(evaluate_prefix_recursive(bindings, expression_a))  # Output: 0
print(evaluate_prefix_recursive(bindings, expression_b))  # Output: 1
print(evaluate_prefix_recursive(bindings, expression_c))  # Output: 0

0
1
0


# QUESTION 4 (EXPECTED)


Using the recursive prefix evaluator you defined in the previous question
answer the following question (you will need to convert the exressions below
to prefix). You can use multiple string assignments to assemble more complicated
sentences into one big string:


Let A be the formula:

\begin{equation}
  (( p_{1} \rightarrow (p2 \land p_{3})) \land ((\neg p_{1})
  \rightarrow (p_{3} \land p_{4})))
\end{equation}

Let B be the formula:

\begin{equation}
  (( p_{3} \rightarrow (\neg p_{6})) \land ((\neg
  p_{3}) \rightarrow (p_{4} \rightarrow p_{1})))  
\end{equation}

Let C be the formula:

\begin{equation}
  ((\neg(p2 \land p_{5})) \land (p2 \rightarrow p_{5}))
\end{equation}

Let D be the formula:

\begin{equation}
  (\neg (p_{3} \rightarrow p_{6}))
\end{equation}

Evaluate the formulate E:
\begin{equation}
  (( A \land (B \land C)) \rightarrow D)
\end{equation}

under the true assignment $I_{1}$, where $I_{1}(p_{1}) = I_{1}(p_{3}) = I_{1}(p_{5}) = false$
and $I_{1}(p2) = I_{1}(p_{4}) = I_{1}(p_{6}) = true$ as well as under the truth assignment
$I_{2}$, where $I_{2}(p_{1}) = I_{2}(p_{3}) = I_{2}(p_{5}) = true$ and
$I_{2}(p_{2})=I_{2}(p_{4})=I_{2}(p_{6}) = false$.


In [90]:
## YOUR CODE GOES HERE

# I1 = {
#     'p1': False,
#     'p3': False,
#     'p5': False,
#     'p2': True,
#     'p4': True,
#     'p6': True
# }

# I2 = {
#     'p1': True,
#     'p3': True,
#     'p5': True,
#     'p2': False,
#     'p4': False,
#     'p6': False
# }

# # Define the logical expressions A, B, C, D, and E in prefix notation
# expression_A = "& => p1 & p2 p3 => ~p1 & p3 p4"
# expression_B = "& => p3 ~p6 => ~p3 => p4 p1"
# expression_C = "& ~ & p2 p5 => p2 p5"
# expression_D = "~ => p3 p6"
# expression_E = "(=> (& A (& B C)) D)"

# # Evaluate formula E using the defined prefix expressions and truth assignments
# result_I1 = evaluate_prefix_recursive(I1, expression_E.replace('A', expression_A).replace('B', expression_B).replace('C', expression_C).replace('D', expression_D))
# result_I2 = evaluate_prefix_recursive(I2, expression_E.replace('A', expression_A).replace('B', expression_B).replace('C', expression_C).replace('D', expression_D))

# # Display the results
# print(f"Result under truth assignment I1: {result_I1}")  # Output: Result under truth assignment I1: 1
# print(f"Result under truth assignment I2: {result_I2}")  # Output: Result under truth assignment I2: 1



ValueError: ignored

# QUESTION 5 (ADVANCED)

Implement inference using model-checking using your prefix recursive evaluator to decide whether a knowledge base KB entais some sentence a. To do so express the knowledge base in the prefix notation, enumerate all models for the variables in the dictionary, and check that the sentence a is true in every model in which the KB is true.

You can check the implementation to tt_entails in logic.ipynb in the aima_python repository to inform how you implement your solution. Your solution should NOT rely directly on any code in logic.py or logic.ipynb.

Check you model checking by showing whether A & (B | C) & D & E & (~F & ~G) entails A & D & E & ~F & ~G.
You will need to convert this example to prefix notation. It is also a good idea to check a
few simple cases to confirm that your approach works.


In [ ]:
# YOUR CODE GOES HERE


# Extra ideas (no credit)

* Implement conversion of the prefix expressions to prefix conjuctive normal form (CNF) based on the recursive evaluator you have implemented.
* Based on the recursive evaluator you have implemented do a conversion of expressions in prefix notation to the infix notation of expressions supported by logic.ipynb. Provide 4 test cases that demonstrate the the conversion works by confirming that the result of your evaluator and the logic.ipynb evaluator are the same.



# Question 6 (Basic)

Consider the following propositional logic knowledge base.

It is not sunny this afternoon and it is colder than yesterday.
We will go swimming only if it is sunny.
If we do not go swimming then we will take a canoe trip.
If we take a canoe trip, then we will be home by sunset.
Denote:

* p = It is sunny this afternoon
* q = it is colder than yesterday
* r = We will go swimming
* s= we will take a canoe trip
* t= We will be home by sunset

Express this knowledge base using propositional logic using the expression syntax used in logic.ipynb. You can incoprorate any code you need from logic.ipynb and logic.py. Using both model checking and theorem proving inference (you can use the implementations providedin logic.py) show that this knowledge base entails the sentence if it is not sunny this afternoon then we will be home by sunset.

In [ ]:
# YOUR CODE GOES HERE



# Question 7 (Basic)  

Encode the kindship domain described in section 8.3.2 of the textbook using FOL and FolKB implementation in logic.py and encode as facts the relationships between the members of the Simpsons family from the popular TV show:  

https://en.wikipedia.org/wiki/Simpson_family


Show how the following queries can be answered using this KB:

* Who are the children of Homer ?
* Who are the parents of Bart ?
* Are Lisa and Homer siblings ?
* Are Lisa and Bart siblings ?


In [ ]:
# YOUR CODE GOES HERE

# QUESTION 8 (EXPECTED)

In this question we explore Prolog which is a programming language based on logic. We won't go into details but just wanted to give you a flavor of the syntax and how it connects to what we have learned. For this question you
will NOT be using the notebook so your answer should just be the source code. We will use http://tau-prolog.org/ which is a Prolog implementation that can run in a browser. When you access the webpage there is a text window labeled try it for entering your knowledge base and under it t
here is a text entry field for entering your query.

For example type in the Try it window and press enter:

```Prolog
likes(sam, salad).
likes(dean, pie).
likes(sam, apples).
likes(dean, whiskey).
```

Then enter the query:
```Prolog
likes(sam,X).
```
When you press Enter once you will get X=apples. and X=salad. Note the periods at the end of each statement.

Encode the kinship domain from the previous question in Prolog and answer the queries from the previous question. Notice that in Prolog the constants start with lower case letters and the variables start with upper case letters.

Provide your code for the KB and queries using markup. See the syntax for Prolog of this cell by double clicking for editing.



In [ ]:
# YOUR CODE GOES HERE (USING MARKDOWN)

# QUESTION 9 (EXPECTED)


## Legoworld


In this question we explore the use of FOL to encode knowledge about the objects
and arrangement of a simple world created by different lego pieces. Our world
will consist of making simple structure by placing lego pieces on top of each other.
Each lego piece will be identified by a unique identifier (the letters in the figure below).

Let's look at a specific example where each piece is labeled by a letter:

<img src="https://github.com/alexwholland/CSC-421/blob/main/Assignment%202/lego_letters.png?raw=1" width="60%"/>

This corresponds to the following picture:

<img src="https://github.com/alexwholland/CSC-421/blob/main/Assignment%202/lego2.png?raw=1" width="60%"/>



We can use the following predicates to model the world:
* OnPlate(p): p is attached to the bottom plate
* On(p1,p2): piece p1 is placed on top of piece p2
* AtLeft(p1,p2): piece p1 and piece p2 are placed on the plate, and piece p1 is direct at the left of p2
* Color(p,c): The color of piece p is c (Red, Grey, Brown, White, Yellow, Blue)
* Type(p,t): The type of piece p is t (Brick, Plate, Tile)

<img src="https://github.com/alexwholland/CSC-421/blob/main/Assignment%202/lego3.png?raw=1" width="20%"/>


Each pieces will be identified by the letters appearing in the picture. The thicker brick with studs will have type  Brick, the thinner brick with studs is of type Plate, and the one that is flat on the top is of type Tile.


Use the FO KB implementation in logic.py to:

1. Write a database of facts which models the world in the picture. For example you can use clauses.append(expr('TypeOf(A,Brick)')) to state that lego piece A is a Brick.

2. Based exclusively on using these predicates (OnPlate, On, AtLeft), define the following predicates:
    * Base(b1, b2): b2 is the base of the tower containing b1.
    * Base_at_right(b1, b2): b1 and b2 are on the plate, and b2 is at the right (but perhaps not directly) of b1.
    * Object_at_right(b1, b2): b1 is in a pile which is at the right (but perhaps not directly) of b2.
    
    
The above predicates must work for any world defined using the facts specified by on_plate, on,
at_left not just the specific example encoded above. In other words these predicates should be defined
in terms of the existing predicates and variables. As an example here is the definition of Base.

    * clauses.append(expr('OnPlate(x) ==> Base(x,x)'))
    * clauses.append(expr('On(x,z) & Base (z,y) ==> Base(x,y)'))
    
This is a recursive definition it is a good idea to see how it works by doing substitutions by hand.


3. Using the KB you created answer the following queries:
    * Is piece B on top of piece C?
    * What is the type and color of the piece on top of C?
    * What is the type of the base of H?
    * What are the bricks that are right of C?
    * What are all the bricks that are on top of i ?






# QUESTION 10 (ADVANCED)


This question is more advanced and open ended. I provide two options. You only need to implement one
of the two options to get full credit for this question. You are welcome to implement both but
you will still get 1 point.

## Option 1

Extend the Legoworld knowledge base with a predicate to determine if a brick is part of an unstable
tower. Any brick placed on top of a tile results in an unstable tower. For example the brown plate
and grey brick in the middle are unstable but the red brick under the tile in the middle is stable.
This is a trickier predicate to define so show a few cases to ensure that it works as expected.

<img src="https://github.com/alexwholland/CSC-421/blob/main/Assignment%202/lego1.png?raw=1" width="40%"/>

Provide a simple natural language processing interface to the LegoWorld that takes input from
the user and returns the results in more natural lanuage. For example you could have the following
dialog:

* User: What color is brick B ?
* Computer: Brick B is Red.
* User: Is there a brick that is on top of a tile?
* Computer: Yes, brick D is on top of a tile.
etc

You will basically write a simple translation layer from simplified English to FOL tell and ask requests and back to simplified English.


This question is inspired by the classic natural understanding work using logic: https://en.wikipedia.org/wiki/SHRDLU



## Option 2


This question explores the automatic constructions of a first-order logic knowledge base from a web resource and is more open ended than the other ones. The website https://www.songfacts.com/ contains a large variety of facts about music. Check the https://www.songfacts.com/categories link for some categories. Using selenium Python bindings https://selenium-python.readthedocs.io/ access the webpage and scrape at least three categories. Your code should scrape the information from the pages and convert it into relationships and facts in first-order logic using the syntax of expressions in logic.ipynb. Once you build your knowledge-base then write 4 non-trivial queries that show-case the expressiveness of FOL. These queries should not be possible to be answered easily using the web interface i.e they should have some logical connectives, more than one possible answer etc. The translation of the song facts from the web page to FOL should NOT be done by hand but using the web scraping tool you develop. You can use multiple cells in your answer.